# Importing necassary libraries

In [1]:
import numpy as np
import scipy.io
from python_speech_features import mfcc
import matplotlib.pyplot as plt
import IPython.display as ipd
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import soundfile as sf
import random
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import joblib
import math
import statistics




# Defining the function that reads the audios from a given path using soundfile and returns 4 lists : audios, freqs, filepaths and problematic files for potential debugging

In [2]:

def read_audios(directory):
    audios = []
    freqs = []
    filepaths = []
    problematic_files = []
    
    # Walking through the directory that contains the dataset and reading each file that has the .wav extension
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                filepath = os.path.join(root, file)
                filepaths.append(filepath)
                try:
                    # Read the audio file using soundfile
                    data, freq = sf.read(filepath)
                    audios.append(data)
                    freqs.append(freq)
                except Exception as e:
                    print(f"Encountered an error with file: {filepath}. Error: {e}")
                    problematic_files.append(filepath)
    
    return audios, freqs, filepaths, problematic_files

# Example usage
audios, freqs, filepaths, problematic_files = read_audios(r'italian_wav')


# Defining the function that extracts the mfcc features then removes the frames of silence finally it saves the mffc features into a .txt file according to gender 

In [3]:
def extractMfccs_RemoveSilence_saveMfccs(audios,freqs,filepaths, directory):
    mfccs = []
 
    for audio, freq, filepath in zip(audios, freqs,filepaths):
        # extract the MFCC features
        mfcc_features = mfcc(audio, freq, winlen=0.025, winstep=0.01, numcep=13, nfilt=26, nfft= 2048, lowfreq=0,
                         highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=False)
        
        # calculate the energy
        energy = np.sum(mfcc_features**2, axis=1)
        # calculate the threshold for silence
        threshold = np.mean(energy) * 0.4
        #removing silence frames from mfccs
        voiced_indices = np.where(energy > threshold)[0]
        mfccs_voiced = mfcc_features[voiced_indices,:]
        mfccs.append(mfccs_voiced)
        
        # print the shape of the MFCCs before and after removing silence
        print(f"MFCCs before removing silence: {mfcc_features.shape}")
        print(f"MFCCs after removing silence: {mfccs_voiced.shape}")
        
       #saving mffcs 
       # extract the gender information from the file name
        gender = None
        if 'hommes' in filepath:
            gender = 'Hommes'
        elif 'femmes' in filepath:
            gender = 'Femmes'

        # save the MFCCs to the appropriate directory based on gender
        if gender is not None:
            gender_dir = os.path.join(directory, gender)
            if not os.path.exists(gender_dir):
                os.makedirs(gender_dir)
            mfcc_file = os.path.join(gender_dir,  os.path.splitext(os.path.basename(filepath))[0] + ".mfcc")
            np.savetxt(mfcc_file, mfccs_voiced, delimiter=',')
            
      
    
    return  mfccs

# Defining the function that splits the extraced mfccs into training and testing sets : 2/3 from male directory for training and 2/3 from female 

In [4]:
def train_test_split(mfcc_dir):
    # create separate lists for male and female file paths
    male_files = []
    female_files = []
    for root, dirs, files in os.walk(mfcc_dir):
        for file in files:
            if file.endswith('.mfcc'):
                if 'Hommes' in root:
                    male_files.append(os.path.join(root, file))
                elif 'Femmes' in root:
                    female_files.append(os.path.join(root, file))

    # shuffle the male and female lists independently
    random.shuffle(male_files)
    random.shuffle(female_files)

    # split the male and female lists into train and test based on the desired ratio
    male_train = male_files[:int(2/3*len(male_files))]
    male_test = male_files[int(2/3*len(male_files)):]

    female_train = female_files[:int(2/3*len(female_files))]
    female_test = female_files[int(2/3*len(female_files)):]

    
    
    # merge the train and test lists for both male and female
    train_files = male_train + female_train
    test_files = male_test + female_test

    # load the MFCC features from the saved files for the train and test sets
    train_mfccs = []
    test_mfccs = []

    for file in train_files:
        train_mfccs.append(np.loadtxt(file, delimiter=','))

    for file in test_files:
        test_mfccs.append(np.loadtxt(file, delimiter=','))

    # print the shapes of the train and test MFCC feature arrays
    print(f"Train male MFCCs shape: {np.array(male_train).shape}")
    print(f"Test male MFCCs shape: {np.array( male_test).shape}")
    print(f"Train female MFCCs shape: {np.array(female_train).shape}")
    print(f"Test female MFCCs shape: {np.array( female_test).shape}")
    print(f"Train MFCCs shape: {np.array(train_mfccs).shape}")
    print(f"Test MFCCs shape: {np.array(test_mfccs).shape}")
    
    return train_mfccs, test_mfccs


# Defining the functions that train the different GMM models and than save them as a pkl file

In [5]:
def gmm16(train_mfccs):
    # Initialize the GMM model with 16 classes
    gmm = GaussianMixture(n_components=16, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model16_italian.pkl')

    return gmm


In [6]:
def gmm32(train_mfccs):
    # Initialize the GMM model with 32 classes
    gmm = GaussianMixture(n_components=32, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model32_italian.pkl')

    return gmm

In [7]:
def gmm64(train_mfccs):
    # Initialize the GMM model with 64 classes
    gmm = GaussianMixture(n_components=64, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model64_italian.pkl')

    return gmm

In [8]:
def gmm128(train_mfccs):
    # Initialize the GMM model with 128 classes
    gmm = GaussianMixture(n_components=128, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model128_italian.pkl')

    return gmm

In [9]:
def gmm256(train_mfccs):
    # Initialize the GMM model with 254 classes
    gmm = GaussianMixture(n_components=256, covariance_type='diag', random_state=0)

    # Fit the GMM model to the training data
    gmm.fit(train_mfccs)
    
    # Save the trained GMM model to a file
    joblib.dump(gmm, r'gmm_model256_italian.pkl')

    return gmm

# Getting the audios frequencies and filepaths from their directory using the function defined above

In [10]:
audios, freqs, filepaths, problematic_files = read_audios(r'italian_wav')


# Extracting the mfcc features - Removing silence and saving the mfccs

In [11]:
mfccs = extractMfccs_RemoveSilence_saveMfccs(audios, freqs, filepaths,r'mfcc_italian')

MFCCs before removing silence: (186, 13)
MFCCs after removing silence: (185, 13)
MFCCs before removing silence: (443, 13)
MFCCs after removing silence: (442, 13)
MFCCs before removing silence: (925, 13)
MFCCs after removing silence: (920, 13)
MFCCs before removing silence: (376, 13)
MFCCs after removing silence: (376, 13)
MFCCs before removing silence: (424, 13)
MFCCs after removing silence: (419, 13)
MFCCs before removing silence: (481, 13)
MFCCs after removing silence: (480, 13)
MFCCs before removing silence: (378, 13)
MFCCs after removing silence: (378, 13)
MFCCs before removing silence: (419, 13)
MFCCs after removing silence: (416, 13)
MFCCs before removing silence: (345, 13)
MFCCs after removing silence: (345, 13)
MFCCs before removing silence: (484, 13)
MFCCs after removing silence: (483, 13)
MFCCs before removing silence: (837, 13)
MFCCs after removing silence: (835, 13)
MFCCs before removing silence: (726, 13)
MFCCs after removing silence: (726, 13)
MFCCs before removing silenc

MFCCs before removing silence: (333, 13)
MFCCs after removing silence: (333, 13)
MFCCs before removing silence: (232, 13)
MFCCs after removing silence: (232, 13)
MFCCs before removing silence: (251, 13)
MFCCs after removing silence: (251, 13)
MFCCs before removing silence: (249, 13)
MFCCs after removing silence: (248, 13)
MFCCs before removing silence: (189, 13)
MFCCs after removing silence: (189, 13)
MFCCs before removing silence: (263, 13)
MFCCs after removing silence: (263, 13)
MFCCs before removing silence: (297, 13)
MFCCs after removing silence: (297, 13)
MFCCs before removing silence: (385, 13)
MFCCs after removing silence: (385, 13)
MFCCs before removing silence: (265, 13)
MFCCs after removing silence: (265, 13)
MFCCs before removing silence: (424, 13)
MFCCs after removing silence: (424, 13)
MFCCs before removing silence: (225, 13)
MFCCs after removing silence: (225, 13)
MFCCs before removing silence: (261, 13)
MFCCs after removing silence: (261, 13)
MFCCs before removing silenc

MFCCs before removing silence: (923, 13)
MFCCs after removing silence: (919, 13)
MFCCs before removing silence: (625, 13)
MFCCs after removing silence: (625, 13)
MFCCs before removing silence: (678, 13)
MFCCs after removing silence: (677, 13)
MFCCs before removing silence: (270, 13)
MFCCs after removing silence: (270, 13)
MFCCs before removing silence: (558, 13)
MFCCs after removing silence: (558, 13)
MFCCs before removing silence: (304, 13)
MFCCs after removing silence: (304, 13)
MFCCs before removing silence: (268, 13)
MFCCs after removing silence: (268, 13)
MFCCs before removing silence: (294, 13)
MFCCs after removing silence: (293, 13)
MFCCs before removing silence: (409, 13)
MFCCs after removing silence: (409, 13)
MFCCs before removing silence: (642, 13)
MFCCs after removing silence: (642, 13)
MFCCs before removing silence: (505, 13)
MFCCs after removing silence: (505, 13)
MFCCs before removing silence: (273, 13)
MFCCs after removing silence: (273, 13)
MFCCs before removing silenc

MFCCs before removing silence: (745, 13)
MFCCs after removing silence: (745, 13)
MFCCs before removing silence: (565, 13)
MFCCs after removing silence: (565, 13)
MFCCs before removing silence: (628, 13)
MFCCs after removing silence: (627, 13)
MFCCs before removing silence: (791, 13)
MFCCs after removing silence: (790, 13)
MFCCs before removing silence: (613, 13)
MFCCs after removing silence: (611, 13)
MFCCs before removing silence: (909, 13)
MFCCs after removing silence: (909, 13)
MFCCs before removing silence: (534, 13)
MFCCs after removing silence: (534, 13)
MFCCs before removing silence: (582, 13)
MFCCs after removing silence: (582, 13)
MFCCs before removing silence: (844, 13)
MFCCs after removing silence: (844, 13)
MFCCs before removing silence: (825, 13)
MFCCs after removing silence: (825, 13)
MFCCs before removing silence: (601, 13)
MFCCs after removing silence: (601, 13)
MFCCs before removing silence: (628, 13)
MFCCs after removing silence: (628, 13)
MFCCs before removing silenc

MFCCs before removing silence: (899, 13)
MFCCs after removing silence: (891, 13)
MFCCs before removing silence: (412, 13)
MFCCs after removing silence: (411, 13)
MFCCs before removing silence: (813, 13)
MFCCs after removing silence: (807, 13)
MFCCs before removing silence: (251, 13)
MFCCs after removing silence: (250, 13)
MFCCs before removing silence: (196, 13)
MFCCs after removing silence: (193, 13)
MFCCs before removing silence: (275, 13)
MFCCs after removing silence: (273, 13)
MFCCs before removing silence: (652, 13)
MFCCs after removing silence: (650, 13)
MFCCs before removing silence: (268, 13)
MFCCs after removing silence: (268, 13)
MFCCs before removing silence: (765, 13)
MFCCs after removing silence: (754, 13)
MFCCs before removing silence: (702, 13)
MFCCs after removing silence: (701, 13)
MFCCs before removing silence: (263, 13)
MFCCs after removing silence: (263, 13)
MFCCs before removing silence: (335, 13)
MFCCs after removing silence: (328, 13)
MFCCs before removing silenc

MFCCs before removing silence: (805, 13)
MFCCs after removing silence: (802, 13)
MFCCs before removing silence: (983, 13)
MFCCs after removing silence: (982, 13)
MFCCs before removing silence: (349, 13)
MFCCs after removing silence: (349, 13)
MFCCs before removing silence: (762, 13)
MFCCs after removing silence: (761, 13)
MFCCs before removing silence: (685, 13)
MFCCs after removing silence: (682, 13)
MFCCs before removing silence: (849, 13)
MFCCs after removing silence: (848, 13)
MFCCs before removing silence: (769, 13)
MFCCs after removing silence: (768, 13)
MFCCs before removing silence: (861, 13)
MFCCs after removing silence: (858, 13)
MFCCs before removing silence: (234, 13)
MFCCs after removing silence: (234, 13)
MFCCs before removing silence: (304, 13)
MFCCs after removing silence: (302, 13)
MFCCs before removing silence: (661, 13)
MFCCs after removing silence: (660, 13)
MFCCs before removing silence: (277, 13)
MFCCs after removing silence: (277, 13)
MFCCs before removing silenc

MFCCs before removing silence: (397, 13)
MFCCs after removing silence: (397, 13)
MFCCs before removing silence: (239, 13)
MFCCs after removing silence: (239, 13)
MFCCs before removing silence: (808, 13)
MFCCs after removing silence: (807, 13)
MFCCs before removing silence: (364, 13)
MFCCs after removing silence: (363, 13)
MFCCs before removing silence: (361, 13)
MFCCs after removing silence: (359, 13)
MFCCs before removing silence: (253, 13)
MFCCs after removing silence: (252, 13)
MFCCs before removing silence: (354, 13)
MFCCs after removing silence: (353, 13)
MFCCs before removing silence: (774, 13)
MFCCs after removing silence: (773, 13)
MFCCs before removing silence: (285, 13)
MFCCs after removing silence: (285, 13)
MFCCs before removing silence: (882, 13)
MFCCs after removing silence: (882, 13)
MFCCs before removing silence: (493, 13)
MFCCs after removing silence: (488, 13)
MFCCs before removing silence: (289, 13)
MFCCs after removing silence: (289, 13)
MFCCs before removing silenc

## -----> We can see here that the size of the mfcc features has decreased after removing the frames ot silence

# Splitting into test and train sets according to gender

In [12]:
train_mfccs, test_mfccs = train_test_split(r'mfcc_italian')

Train male MFCCs shape: (198,)
Test male MFCCs shape: (100,)
Train female MFCCs shape: (236,)
Test female MFCCs shape: (118,)
Train MFCCs shape: (434,)
Test MFCCs shape: (218,)


C:\Users\HP\AppData\Local\Temp\ipykernel_6160\3397580047.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f"Train MFCCs shape: {np.array(train_mfccs).shape}")
C:\Users\HP\AppData\Local\Temp\ipykernel_6160\3397580047.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f"Test MFCCs shape: {np.array(test_mfccs).shape}")


# Stacking vertically the train and test MFCC features so that we can fit the gmm models

In [13]:
#stack vertically the train MFCC features 
mfcc_train = []
for train_mfcc in train_mfccs:
    mfcc_train.append(train_mfcc)
mfcc_train = np.concatenate(mfcc_train, axis=0)

#stack vertically the test MFCC features 
mfcc_test = []
for test_mfcc in test_mfccs:
    mfcc_test.append(test_mfcc)
mfcc_test = np.concatenate(mfcc_test, axis=0)


# Saving the test set into a txt file 

In [15]:
#Save the test mfccs in a file
test_mfccs = np.vstack(test_mfccs)
test_mfccs = np.array(test_mfccs, dtype=float)
np.savetxt(r'italianTest', test_mfccs, delimiter=',')



In [16]:
mfcc_train.shape

(211298, 13)

# Training the different Gmm Models 

In [17]:
gmm16 = gmm16(mfcc_train)

In [18]:
gmm32 = gmm32(mfcc_train)

In [19]:
gmm64 = gmm64(mfcc_train)

In [20]:
gmm128= gmm128(mfcc_train)

In [21]:
gmm256= gmm256(mfcc_train)

# Evaluate the performance of each GMM model on the test set using the score_samples() function that returns an array containing the log-likelihood of each frame of the mfcc features

In [22]:
scores = []
for model in [gmm16, gmm32, gmm64, gmm128, gmm256]:
    score = model.score_samples(mfcc_test)
    scores.append(score)

# Print the scores
print('GMM16 score:', scores[0])
print('GMM32 score:', scores[1])
print('GMM64 score:', scores[2])
print('GMM128 score:', scores[3])
print('GMM256 score:', scores[4])



GMM16 score: [-61.06854056 -62.42582096 -62.70123699 ... -52.55981445 -57.02021069
 -55.0678287 ]
GMM32 score: [-56.22213453 -58.17503175 -60.48209144 ... -53.02644479 -56.7456003
 -60.66568073]
GMM64 score: [-59.62345905 -60.31933487 -60.50595392 ... -53.71247843 -56.88883863
 -60.95527587]
GMM128 score: [-55.8549394  -58.80908677 -62.17018931 ... -53.60496978 -56.14766443
 -57.68058542]
GMM256 score: [-57.55851719 -60.16487996 -61.64768258 ... -54.82653241 -56.79610362
 -53.02334307]


# Comparing the size of our mfcc_test set with the size of the scores array 

In [23]:
mfcc_test.shape

(108072, 13)

# Indeed the size of the scores array is the same 

In [24]:
 scores[0].shape

(108072,)

# In order to compare between the different GMM Models we need to calculate the score for the whole test set and we can do that by calculating the mean of the individual scores

In [25]:
#calculationg the score of the hole test set
print('GMM16 score:', scores[0].mean())
print('GMM32 score:', scores[1].mean())
print('GMM64 score:', scores[2].mean())
print('GMM128 score:', scores[3].mean())
print('GMM256 score:', scores[4].mean())


GMM16 score: -53.822338694542395
GMM32 score: -53.15839096247776
GMM64 score: -52.64062143574135
GMM128 score: -52.22434124796724
GMM256 score: -51.87520019256102


### From the results above we can see that the best score (the closest one to 0) is given by the model using 16 gaussians 